# FathomNet Workshop
*So you want to use FathomNet data...*

<img src="https://raw.githubusercontent.com/fathomnet/fathomnet-logo/main/FathomNet_white_CenterText_400px.png" alt="FathomNet logo" width="200"/>

## Introduction

TODO
- Organize
- Expand on what's on the FathomNet GitHub overall
- Brief overview of what fathomnet-py does
- Highlight some use cases?
- Discuss what to expect in this notebook

[FathomNet GitHub](https://github.com/fathomnet)

[fathomnet-py](https://github.com/fathomnet/fathomnet-py)
[docs](https://fathomnet-py.readthedocs.io)

fathomnet-py is designed to help scientists, researchers, and developers interact with FathomNet data. 

The fathomnet-py API offers native Python interaction with the FathomNet REST API, abstracting away the underlying HTTP requests.

### Installing `fathomnet-py`

To install fathomnet-py, you will need to have Python 3.7 or greater installed first. Then, from the command-line:

```bash
pip install fathomnet
```

This notebook installs fathomnet-py in the [Setup](#Setup) section, along with some relevant packages for data manipulation and visualization.

## Setup

First, we'll install a few packages via pip:

In [ ]:
!pip install -q -U fathomnet plotly opencv-python ipyleaflet prettyprinter

and import the modules we need for this notebook:

In [ ]:
import ipywidgets as widgets  # Provides Jupyter embedded widgets
import ipyleaflet             # Provides map widgets
import requests               # Manages HTTP requests
import numpy as np            # Facilitates array/matrix operations
import cv2                    # Facilitates image operations
import plotly.express as px   # Generates nice plots
import random                 # Generates pseudo-random numbers
from google.colab.patches import cv2_imshow
import prettyprinter as pp
pp.install_extras(exclude=['django', 'python'])

# fathomnet-py API modules.
# Latest documentation on available modules: 
# https://fathomnet-py.readthedocs.io/en/latest/api.html
from fathomnet.api import images, boundingboxes

## Day 1

### API overview
Condense this.

#### Data types

- Bounding box
- Image
- Tag
- Image set
- Identity
- Darwin core
- Taxa
- Marine region

#### Modules

- `boundingboxes`
- `darwincore`
- `images`
- `geoimages`
- `imagesetuploads`
- `regions`
- `stats`
- `tags`
- `taxa`
- `users`
- `firebase` & `xapikey`

In [ ]:
from fathomnet.api import images

example_image = images.find_by_uuid('79958ac5-832a-488c-9b48-cce7db346497')

pp.pprint(example_image)

### Getting image data & bounding boxes for a concept

First, let's pick a concept that we want to get data for.

To list all the available concepts that have at least 1 bounding box, we can use the `boundingboxes` module:

In [ ]:
# Make a bar chart of the top N concepts by bounding boxes
N = 5

# Get the number of bounding boxes for all concepts
concept_counts = boundingboxes.count_total_by_concept()

# Sort by number of bounding boxes
concept_counts.sort(key=lambda cc: cc.count, reverse=True)

# Get the top N concepts and their counts
concepts, counts = zip(*((cc.concept, cc.count) for cc in concept_counts[:N]))

# Make a bar chart
fig = px.bar(x=concepts, y=counts, labels={'x': 'Concept', 'y': 'Bounding box count'}, text_auto=True)
fig.show()

In [ ]:
# Get a list of all concepts that have at least 1 bounding box
all_concepts = boundingboxes.find_concepts()

# Print how many there are
print('FathomNet has', len(all_concepts), 'concepts!')

# Pick one!
concept_combo = widgets.Combobox(
    options=all_concepts,
    description='Concept:',
    placeholder='Double-click or type here',
    ensure_option=True,
    disabled=False
)
concept_combo

In [ ]:
# Get the selected concept
concept = concept_combo.value
if not concept:
  concept = 'Chionoecetes tanneri'
  print('No concept selected. Using the default:', concept)

# Pull the images from FathomNet
concept_images = images.find_by_concept(concept)

# Print the count
print('Found', len(concept_images), 'images of', concept)

In [ ]:
# Pick a random image
image = concept_images[random.randrange(len(concept_images))]

# Fetch and show the image
image_data = requests.get(image.url).content
widgets.Image(
    value=image_data,
    format='png',
    width=image.width,
    height=image.height,
)

In [ ]:
# Render bounding boxes over the image
buf = np.frombuffer(image_data, np.uint8)
overlay_image = cv2.imdecode(buf, cv2.IMREAD_COLOR)
for bbox in image.boundingBoxes:
  pt1 = bbox.x, bbox.y
  pt2 = bbox.x + bbox.width, bbox.y + bbox.height
  overlay_image = cv2.rectangle(
      overlay_image, 
      pt1, pt2, 
      color=(255, 255, 255),
      thickness=2
  )
  overlay_image = cv2.putText(
      overlay_image,
      bbox.concept + (f'({bbox.altConcept})' if bbox.altConcept is not None else ''),
      (bbox.x, bbox.y - 5),
      color=(255, 255, 255),
      fontFace=cv2.FONT_HERSHEY_PLAIN,
      fontScale=0.05 * image.width ** 0.5,
      thickness=1
  )

cv2_imshow(overlay_image)

### Depth histogram

In [ ]:
# Extract the depth (in meters) from each image
depths = [
    image.depthMeters 
    for image in concept_images 
    if image.depthMeters is not None
]

# Make a horizontal histogram
fig = px.histogram(y=depths, title=f'{concept} images by depth', labels={'y': 'depth (m)'})
fig['layout']['yaxis']['autorange'] = 'reversed'
fig.show()

### Geographic heatmap

In [ ]:
# Extract the latitude/longitude from each image
locations = [
    (image.latitude, image.longitude)
    for image in concept_images
    if image.latitude is not None and image.longitude is not None
]

# Create a map from the Esri Ocean basemap
center = (36.807, -121.988)  # Monterey Bay
map = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.Esri.OceanBasemap, 
    center=center, 
    zoom=10
)
map.layout.height = "800px"

# Overlay the image locations as a heatmap
heatmap = ipyleaflet.Heatmap(
    locations=locations,
    radius=20,
    min_opacity=0.5
)
map.add_layer(heatmap)

map

## Day 2 (Breakout)

### Download images & bounding boxes

### Train a model

### Download a model from the FathomNet model zoo

### Run inference